<a href="https://colab.research.google.com/github/elizabeth-16/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 6.1MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [3]:
import os
import pandas as pd
import numpy as np
df = pd.read_csv('/content/yelp_labelled.csv')
df.head()

,reviews,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
df['label'] = df['label'].replace({1:"good"}).replace({0:"bad"})
df

,reviews,label
0,Wow... Loved this place.,good
1,Crust is not good.,bad
2,Not tasty and the texture was just nasty.,bad
3,Stopped by during the late May bank holiday of...,good
4,The selection on the menu was great and so wer...,good
...,...,...
995,I think food should have flavor and texture an...,bad
996,Appetite instantly gone.,bad
997,Overall I was not impressed and would not go b...,bad
998,"The whole experience was underwhelming, and I ...",bad


In [5]:
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text =re.sub(pattern,'',text)
  return text

In [7]:
df.reviews = df.reviews.apply(lambda x:x.lower())
df.label = df.label.apply(lambda x:x.lower())
df.reviews = df.reviews.apply(remove_stopwords)
df.reviews = df.reviews.apply(remove_sp)
df.head()

,reviews,label
0,wow loved place,good
1,crust not good,bad
2,not tasty texture nasty,bad
3,stopped late may bank holiday rick steve recom...,good
4,selection menu great prices,good


In [8]:
df['compound'] = df['reviews'].apply(lambda x: vs.polarity_scores(x)['compound'])
df.head()

,reviews,label,compound
0,wow loved place,good,0.8271
1,crust not good,bad,-0.3412
2,not tasty texture nasty,bad,0.4449
3,stopped late may bank holiday rick steve recom...,good,0.6908
4,selection menu great prices,good,0.6249


In [9]:
x = df['reviews'].values
y = df['label'].values

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
model = Pipeline([('TFIDF',TfidfVectorizer()),('model',MultinomialNB())])
model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('TFIDF',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [12]:
y_pred = model.predict(x_test)
y_pred

array(['bad', 'bad', 'bad', 'good', 'bad', 'bad', 'good', 'bad', 'bad',
       'good', 'good', 'good', 'bad', 'good', 'good', 'good', 'bad',
       'bad', 'bad', 'good', 'bad', 'bad', 'good', 'good', 'bad', 'good',
       'bad', 'good', 'good', 'bad', 'good', 'good', 'bad', 'bad', 'good',
       'bad', 'bad', 'bad', 'bad', 'good', 'good', 'good', 'good', 'bad',
       'bad', 'bad', 'good', 'good', 'bad', 'bad', 'good', 'bad', 'good',
       'good', 'bad', 'bad', 'good', 'good', 'bad', 'good', 'bad', 'bad',
       'good', 'good', 'good', 'bad', 'good', 'bad', 'good', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'bad', 'bad', 'good', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'good', 'good', 'bad',
       'bad', 'good', 'good', 'bad', 'good', 'bad', 'bad', 'bad', 'bad',
       'good', 'bad', 'bad', 'good', 'bad', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'bad', 'bad', 'bad', 'good', 'bad', 'good',
       'good', 'bad', 'good', 'bad', 'good', 'good', 

In [13]:
y_test

array(['bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'good', 'bad', 'bad',
       'good', 'good', 'good', 'bad', 'good', 'good', 'good', 'bad',
       'bad', 'bad', 'good', 'bad', 'good', 'good', 'bad', 'bad', 'good',
       'good', 'good', 'good', 'bad', 'good', 'good', 'good', 'good',
       'good', 'bad', 'bad', 'bad', 'bad', 'good', 'good', 'bad', 'good',
       'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'good', 'good',
       'good', 'good', 'bad', 'bad', 'good', 'good', 'bad', 'good', 'bad',
       'bad', 'bad', 'bad', 'good', 'bad', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'bad', 'good', 'good',
       'bad', 'good', 'bad', 'good', 'good', 'bad', 'good', 'good', 'bad',
       'bad', 'good', 'bad', 'bad', 'good', 'bad', 'bad', 'bad', 'good',
       'bad', 'good', 'good', 'bad', 'good', 'good', 'good', 'bad',
       'good', 'bad', 'good', 'good', 'bad', 'good', 'good', 'good',
       'bad', 'bad', 'good', 'bad', 'good', 'good', 'good', 'good',

In [14]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy_score(y_pred,y_test)

0.756

In [15]:
confusion_matrix(y_pred,y_test)

array([[91, 35],
       [26, 98]])

In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         bad       0.78      0.72      0.75       126
        good       0.74      0.79      0.76       124

    accuracy                           0.76       250
   macro avg       0.76      0.76      0.76       250
weighted avg       0.76      0.76      0.76       250



In [17]:
import joblib
joblib.dump(model,'Yelp_model')

['Yelp_model']

In [18]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 5.9MB/s 
     |████████████████████████████████| 4.2MB 55.2MB/s 
     |████████████████████████████████| 163kB 54.4MB/s 
     |████████████████████████████████| 112kB 47.0MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 122kB 47.9MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.4 which is incompatible.


In [19]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model = joblib.load('/content/Yelp_model')
st.title('Yelp Classifier')
ip = st.text_input("Enter the message")
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Writing app.py


In [20]:
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://d1fee96a1a8a.ngrok.io'